In [2]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torchvision.models import resnet18, resnet50
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.init as init
import torch.nn.functional as F
import math
import numpy as np
from memory_profiler import memory_usage
import time

In [3]:
# %pip install memory-profiler

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 1

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = torchvision.datasets.ImageFolder(root='/home/sunny/CCRL_vs/Hierarchical_Inference/Datasets/Imagenet1K/val', transform=transform_test)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

model = models.shufflenet_v2_x0_5(pretrained=True)  # Use pretrained weights on ImageNet-1k

model = model.to(device)

model.eval()

num_images = len(test_dataset)  # Get the number of images
values = np.linspace(0, 1, 256)
thresholds = [round(i, 4) for i in values]

num_thresholds = len(thresholds)

def quantize_to_8bit(probabilities):
    # Ensure probabilities are in the [0, 1] range
    probabilities = torch.clamp(probabilities, 0, 1)
    
    # Scale probabilities to 255 intervals, round to the nearest one, and then scale back
    quantized = torch.round(probabilities * 255) / 255
    
    return quantized


/home/sunny/CCRL_vs/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sunny/CCRL_vs/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
mem_before = memory_usage()[0]
start = time.time()
# Initialize the dictionaries for UCB, LCB, Tx, Fx
dict = {
    'UCB': {threshold: 1.0 for threshold in thresholds},  # Upper confidence bound
    'LCB': {threshold: 0.0 for threshold in thresholds},  # Lower confidence bound
    'Tx': {threshold: 0 for threshold in thresholds},      # Count of times each threshold has been reached
    'Fx': {threshold: 0 for threshold in thresholds},    # Fraction of correct predictions
}

# print([dict['UCB'][i] for i in thresholds])

correct_counts = {threshold: 0 for threshold in thresholds}
offloads_counts = {threshold: 0 for threshold in thresholds}
total_counts = {threshold: 0 for threshold in thresholds}
incorrect_counts = {threshold: 0 for threshold in thresholds}
count = 1

# Set tau and delta for thresholding
tau = 0.4  # Example value, set according to your requirements
delta = 1/math.e  # Example value, set according to your requirements
cost = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # print("Image number", count)
        # print()
        
        # Get model outputs and softmax probabilities
        outputs = model(images)
        softmax_probs = F.softmax(outputs, dim=1)

        quantized_probs = quantize_to_8bit(softmax_probs)

        predicted_probs, predicted_labels = torch.max(quantized_probs, dim=1)
        pred_probs = round(predicted_probs.item(), 4)
        # print("SML output", pred_probs)

        pred_class = predicted_labels.item()
        # print("predicted class", pred_class)

        true_class = labels.item()
        # print(pred_class, true_class)

        total_counts[pred_probs] += 1

        if dict['LCB'][pred_probs] >= (1 - tau):
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class is", dict['LCB'][pred_probs])
            # print("Accept, nothing learnt here")
            if pred_class == true_class:
                continue
            else:
                cost += 1
                incorrect_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] <= (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] > (1 - tau) and dict['LCB'][pred_probs] < (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        log_delta = math.log(1/delta)
        uncertainty = math.sqrt(log_delta / (dict['Tx'][pred_probs] + 1e-8))  # Added small value to avoid division by zero

        dict['UCB'][pred_probs] = min(1, dict['Fx'][pred_probs] + uncertainty)
        dict['LCB'][pred_probs] = max(0, dict['Fx'][pred_probs] - uncertainty)
        # print(uncertainty)
        # print(dict['UCB'][pred_probs], dict['LCB'][pred_probs])
        
        for x1 in thresholds:
            if x1 <= pred_probs:
                for x in thresholds:
                    if(x1 <= x and x <= pred_probs):
                        dict['UCB'][x1] = min(dict['UCB'][x1], dict['UCB'][x])

        for x2 in thresholds:
            if x2 >= pred_probs:
                for x in thresholds:
                    if(x2 >= x and x >= pred_probs):
                        dict['LCB'][x2] = max(dict['LCB'][x2], dict['LCB'][x])
                    
        count += 1

end = time.time()

print(f"Time Taken: {end - start}")
mem_after = memory_usage()[0]
print(f"Memory Used: {mem_after - mem_before:.2f} MB")
  
incorrect_sum = 0
total_sum = 0
offload_sum = 0
for i in incorrect_counts:
    incorrect_sum += incorrect_counts[i]
    total_sum += total_counts[i]
    offload_sum += offloads_counts[i]

print(f"Cost: {cost}")

# print(f"Original Accuracy", accuracy)
print(f"Accuracy", (total_sum - incorrect_sum)*100/total_sum)

print(offload_sum, incorrect_sum, total_sum)

Time Taken: 121.49406599998474
Memory Used: 291.59 MB
Cost: 1123.599999999987
Accuracy 93.16849349987254
2139 268 3923


In [11]:
accuracies = [95.18,
97.17,
94.64,
90.03,
96.35,
95.69,
94.82,
96.71,
93.16
]
time_list = [74.47, 72.45, 60.90, 71.87, 121.49]
cost_list = [1129.39, 1111.79, 1124.99, 1123.59]
cost_average = 0.0
average_accuracy = 0
time_average = 0.0
for i in accuracies:
    average_accuracy += i
for i in time_list:
    time_average += i
for i in cost_list:
    cost_average += i

print(average_accuracy/len(accuracies))
print(time_average/len(time_list))
print(cost_average/len(cost_list))

94.8611111111111
80.23600000000002
1122.44


In [6]:
mem_before = memory_usage()[0]

start = time.time()
# Initialize the dictionaries for UCB, LCB, Tx, Fx
dict = {
    'UCB': {threshold: 1.0 for threshold in thresholds},  # Upper confidence bound
    'LCB': {threshold: 0.0 for threshold in thresholds},  # Lower confidence bound
    'Tx': {threshold: 0 for threshold in thresholds},      # Count of times each threshold has been reached
    'Fx': {threshold: 0 for threshold in thresholds},    # Fraction of correct predictions
}

# print([dict['UCB'][i] for i in thresholds])

correct_counts = {threshold: 0 for threshold in thresholds}
offloads_counts = {threshold: 0 for threshold in thresholds}
total_counts = {threshold: 0 for threshold in thresholds}
incorrect_counts = {threshold: 0 for threshold in thresholds}
count = 1

# Set tau and delta for thresholding
tau = 0.4  # Example value, set according to your requirements
delta = 1/math.e  # Example value, set according to your requirements
cost = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # print("Image number", count)
        # print()
        
        # Get model outputs and softmax probabilities
        outputs = model(images)
        softmax_probs = F.softmax(outputs, dim=1)

        quantized_probs = quantize_to_8bit(softmax_probs)

        predicted_probs, predicted_labels = torch.max(quantized_probs, dim=1)
        pred_probs = round(predicted_probs.item(), 4)
        # print("SML output", pred_probs)

        pred_class = predicted_labels.item()
        # print("predicted class", pred_class)

        true_class = labels.item()
        # print(pred_class, true_class)

        total_counts[pred_probs] += 1

        if dict['LCB'][pred_probs] >= (1 - tau):
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class is", dict['LCB'][pred_probs])
            # print("Accept, nothing learnt here")
            if pred_class == true_class:
                continue
            else:
                cost += 1
                incorrect_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] <= (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] > (1 - tau) and dict['LCB'][pred_probs] < (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        log_delta = math.log(1/delta)
        uncertainty = math.sqrt(log_delta / (dict['Tx'][pred_probs] + 1e-8))  # Added small value to avoid division by zero

        dict['UCB'][pred_probs] = min(1, dict['Fx'][pred_probs] + uncertainty)
        dict['LCB'][pred_probs] = max(0, dict['Fx'][pred_probs] - uncertainty)
        
end = time.time()

print(f"Time Taken: {end - start}")

mem_after = memory_usage()[0]
print(f"Memory Used: {mem_after - mem_before:.2f} MB")

incorrect_sum = 0
total_sum = 0
offload_sum = 0
for i in incorrect_counts:
    incorrect_sum += incorrect_counts[i]
    total_sum += total_counts[i]
    offload_sum += offloads_counts[i]

print(f"Cost: {cost}")

# print(f"Original Accuracy", accuracy)
print(f"Accuracy", (total_sum - incorrect_sum)*100/total_sum)

print(offload_sum, incorrect_sum, total_sum)

Time Taken: 41.69509696960449
Memory Used: -4.71 MB
Cost: 1202.1999999999987
Accuracy 98.19016059138414
2828 71 3923


In [12]:
accuracies = [97.57,
97.85,
98.21,
97.96,
98.29,
98.03,
98.16,
98.16,
98.19
]
time_list = [41.24, 41.43, 40.19, 42.43, 40.50, 41.69]
cost_list = [1205.79, 1199.99, 1196.39, 1198.99, 1202.19]
cost_average = 0.0
average_accuracy = 0
time_average = 0.0
for i in accuracies:
    average_accuracy += i
for i in time_list:
    time_average += i
for i in cost_list:
    cost_average += i

print(average_accuracy/len(accuracies))
print(time_average/len(time_list))
print(cost_average/len(cost_list))

98.04666666666665
41.24666666666666
1200.67


In [8]:
mem_before = memory_usage()[0]

start = time.time()

# Algorithm - Hedge
def choose_threshold(weights, thresholds, W_t):
    # Create probability distribution for selecting a threshold
    probabilities = np.array([weights[theta] / W_t for theta in thresholds])
    
    # Choose a threshold based on the weight distribution
    chosen_threshold_idx = np.random.choice(len(thresholds), p=probabilities)
    chosen_threshold = thresholds[chosen_threshold_idx]
    
    return chosen_threshold

# Iterate over different values of alpha
for alpha in np.arange(0, 0.5, 0.4):
    η = 0.02 / alpha**(1/3) if alpha > 0 else 0.02
    ε = min(1, math.sqrt(η / (2 * alpha))) if alpha > 0 else 0.9
    cost = 0

    weights = {threshold: 1.0 for threshold in thresholds}  # Initialize weights for each threshold
    W_t = sum(weights.values())  # Sum the initial weights to get W_t
    incorrect_count = {threshold: 0 for threshold in thresholds}
    correct_count = {threshold: 0 for threshold in thresholds}
    offload_count = {threshold: 0 for threshold in thresholds}
    total_count = {threshold: 0 for threshold in thresholds}
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Get model outputs and softmax probabilities
            outputs = model(images)
            softmax_probs = F.softmax(outputs, dim=1)

            # Quantize probabilities to 8-bit
            quantized_probs = quantize_to_8bit(softmax_probs)

            # Get the predicted probabilities and classes
            predicted_probs, predicted_labels = torch.max(quantized_probs, dim=1)
            p_t = round(predicted_probs.item(), 4)

            pred_class = predicted_labels.item()
            true_class = labels.item()

            # Choose a threshold based on the current weights
            chosen_threshold = choose_threshold(weights, thresholds, W_t)

            total_count[p_t] += 1

            if p_t < chosen_threshold:
                offload_count[p_t] += 1
                cost += tau
            else:
                if pred_class != true_class:
                    incorrect_count[p_t] += 1
                    cost += 1
                else:
                    continue

            # Compute pseudo-loss and update weights
            if pred_class == true_class:
                Y_t = 0
            else:
                Y_t = 1

            for threshold in thresholds:
                if p_t < threshold:
                    pseudo_loss = alpha  # Offload cost
                else:
                    pseudo_loss = Y_t / ε if np.random.binomial(1, ε) else 0  # Loss depends on epsilon

                # Update the weight for the current threshold
                weights[threshold] = weights[threshold] * np.exp(-η * pseudo_loss)

            # Recalculate W_t as the sum of all updated weights
            W_t = sum(weights.values())

end = time.time()

print(f"Time Taken: {end - start}")

mem_after = memory_usage()[0]
print(f"Memory Used: {mem_after - mem_before:.2f} MB")

incorrect_sum = 0
total_sum = 0
offload_sum = 0
for i in incorrect_counts:
    incorrect_sum += incorrect_count[i]
    total_sum += total_count[i]
    offload_sum += offload_count[i]

print(f"Cost: {cost}")

# print(f"Original Accuracy", accuracy)
print(f"Accuracy", (total_sum - incorrect_sum)*100/total_sum)

print(offload_sum, incorrect_sum, total_sum)

Time Taken: 84.75924897193909
Memory Used: 5.77 MB
Cost: 1140.7999999999895
Accuracy 92.30180983940862
2097 302 3923


In [13]:
accuracies = [92.14, 92.58, 92.30]
time_list = [84.01, 84.91, 85.74, 85.08, 83.74, 84.75]
cost_list = [1127.19, 1131.79, 1131.19, 1136.79, 1127.39, 1140.79]
cost_average = 0.0
average_accuracy = 0
time_average = 0.0
for i in accuracies:
    average_accuracy += i
for i in time_list:
    time_average += i
for i in cost_list:
    cost_average += i

print(average_accuracy/len(accuracies))
print(time_average/len(time_list))
print(cost_average/len(cost_list))

92.33999999999999
84.705
1132.5233333333333


In [10]:
mem_before = memory_usage()[0]
start = time.time()
# Initialize the dictionaries for UCB, LCB, Tx, Fx
dict = {
    'UCB': {threshold: 1.0 for threshold in thresholds},  # Upper confidence bound
    'LCB': {threshold: 0.0 for threshold in thresholds},  # Lower confidence bound
    'Tx': {threshold: 0 for threshold in thresholds},      # Count of times each threshold has been reached
    'Fx': {threshold: 0 for threshold in thresholds},    # Fraction of correct predictions
}

# print([dict['UCB'][i] for i in thresholds])

correct_counts = {threshold: 0 for threshold in thresholds}
offloads_counts = {threshold: 0 for threshold in thresholds}
total_counts = {threshold: 0 for threshold in thresholds}
incorrect_counts = {threshold: 0 for threshold in thresholds}
count = 1

# Set tau and delta for thresholding
tau = 0.4  # Example value, set according to your requirements
delta = 1/math.e  # Example value, set according to your requirements
cost = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # print("Image number", count)
        # print()
        
        # Get model outputs and softmax probabilities
        outputs = model(images)
        softmax_probs = F.softmax(outputs, dim=1)

        quantized_probs = quantize_to_8bit(softmax_probs)

        predicted_probs, predicted_labels = torch.max(quantized_probs, dim=1)
        pred_probs = round(predicted_probs.item(), 4)
        # print("SML output", pred_probs)

        pred_class = predicted_labels.item()
        # print("predicted class", pred_class)

        true_class = labels.item()
        # print(pred_class, true_class)

        total_counts[pred_probs] += 1

        if dict['LCB'][pred_probs] >= (1 - tau):
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class is", dict['LCB'][pred_probs])
            # print("Accept, nothing learnt here")
            if pred_class == true_class:
                continue
            else:
                cost += 1
                incorrect_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] <= (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        elif dict['UCB'][pred_probs] > (1 - tau) and dict['LCB'][pred_probs] < (1 - tau):
            dict['Tx'][pred_probs] += 1
            # print("UCB of predicted class", dict['UCB'][pred_probs])
            # print("LCB of predicted class", dict['LCB'][pred_probs])
            # print("Offload")
            if pred_class == true_class:
                correct_counts[pred_probs] += 1
            dict['Fx'][pred_probs] = correct_counts[pred_probs] / dict['Tx'][pred_probs]
            cost += tau
            offloads_counts[pred_probs] += 1

        log_delta = math.log(1/delta)
        uncertainty = math.sqrt(log_delta / (dict['Tx'][pred_probs] + 1e-8))  # Added small value to avoid division by zero

        dict['UCB'][pred_probs] = min(1, dict['Fx'][pred_probs] + uncertainty)
        dict['LCB'][pred_probs] = max(0, dict['Fx'][pred_probs] - uncertainty)
        # print(uncertainty)
        # print(dict['UCB'][pred_probs], dict['LCB'][pred_probs])
        
        eta = max(0.2, min(0.25, uncertainty))  # Dynamically adjust epsilon

        for x1 in thresholds:
            if x1 <= pred_probs:
                for x in thresholds:
                    if x1 <= x and x <= pred_probs:
                        if uncertainty < eta:
                            dict['UCB'][x1] = min(dict['UCB'][x1], dict['UCB'][x])

        for x2 in thresholds:
            if x2 >= pred_probs:
                for x in thresholds:
                    if x2 >= x and x >= pred_probs:
                        if uncertainty < eta:
                            dict['LCB'][x2] = max(dict['LCB'][x2], dict['LCB'][x])

                    
        count += 1

end = time.time()

print(f"Time Taken: {end - start}")
mem_after = memory_usage()[0]
print(f"Memory Used: {mem_after - mem_before:.2f} MB")
  
incorrect_sum = 0
total_sum = 0
offload_sum = 0
for i in incorrect_counts:
    incorrect_sum += incorrect_counts[i]
    total_sum += total_counts[i]
    offload_sum += offloads_counts[i]

print(f"Cost: {cost}")

# print(f"Original Accuracy", accuracy)
print(f"Accuracy", (total_sum - incorrect_sum)*100/total_sum)

print(offload_sum, incorrect_sum, total_sum)

Time Taken: 61.033531665802
Memory Used: -3.61 MB
Cost: 1194.199999999997
Accuracy 97.88427224063217
2778 83 3923


In [5]:
accuracies = [98.44, 97.75, 97.83, 97.93, 98.08]
time_list = [61.51, 60.95, 60.95, 61.13, 60.22]
cost_list = [1201.79, 1198.79, 1192.59, 1196.59, 1196.19]
cost_average = 0.0
average_accuracy = 0
time_average = 0.0
for i in accuracies:
    average_accuracy += i
for i in time_list:
    time_average += i
for i in cost_list:
    cost_average += i

print(average_accuracy/len(accuracies))
print(time_average/len(time_list))
print(cost_average/len(cost_list))

97.9875
61.135000000000005
1197.44


eta = max(0.1, min(0.05, uncertainty))  # Dynamically adjust epsilon
<br>
Time Taken: 61.69447326660156    <br>
Memory Used: -11.68 MB    <br>
Cost: 1195.7999999999959    <br>
Accuracy 97.88427224063217    <br>
2782 83 3923
<br>